# Pade_approximants
A short Pade program, performing analytic continuation using an average of different Pade approximants. 

Given input data $f(z_{in})$ and $z_{out}$, the output data is $f(z_{out})$. 

Features:
- Double precision is used (note: sometimes at least quadruple precision is needed).
- Enforcing mirror symmetry $f(z^*)=f(z)^*$ of some of the input data is possible, look at variable 'nmins'. 
- Several Pade approximants are used and several subsets of the input data are continued. 
- An averaging of the the continuations are done.
- Rejects continuations with positive imaginary part.

### Calculation of Padé coefficients and evaluation on points in the complex plane
The Padé approximant is written as
$P(z) = \frac{a_0+a_1 z + a_2 z^2 + ... + a_{r-1} z^{r-1}}{b_0 + b_1 z +b_2 z^2 + ... + b_{r-1} z^{r-1} + z^r } = \frac{\sum_{i=0}^{r-1} a_i z^i}{\sum_{j=0}^{r-1} b_j z^j + z^r}$
where $N=2r$ is the total number of Padé coefficients.
$P(z)$ can be evaluated everywhere in the complex plane.
Note that in the PRB paper "Analytic continuation by averaging Padé approximants" and in one Fortran implementation the convention is instead with indices starting with 1 instead of zero. 

The coefficients are calculated by setting the Padé approximant equal to the $M$ input data points $f(z)$.
This implies the equation
$f(z) ( b_0 + b_1 z +b_2 z^2 + ... + b_{r-1} z^{r-1} + z^r )  = a_0+a_1 z + a_2 z^2 + ... + a_{r-1} z^{r-1} $
which can be rewritten to

$a_0+a_1 z + a_2 z^2 + ... + a_{r-1} z^{r-1} - f(z) ( b_0 + b_1 z +b_2 z^2 + ... + b_{r-1} z^{r-1} )  = f(z) z^r $

In matrix form this becomes:

$A x = y$, with

$y = f(z) z^r$,

$x = [a_0, a_1 , ..., a_{r-1}, b_0, b_1, ..., b_{r-1}]$ and

$A(i,:) = [ 1 , z , ...., z^{r-1} , -f(z) , -f(z) z , ..., -f(z) z^{r-1} ]$

We can solve this in a least square sense.

The Padé coefficients are then used to calculate P on the disired points in the complex plane. 

The function acPade below is doing this.

### Averaging 
The value of the Pade approximants on the disired points in the complex plane are stored.
The parameters $N$,$M$ are varied. All the output are stored in variable 'fs'.
The distance between the imaginary parts of the continuations are calculated. 

$\Delta_{i,j} = \sum_k |\mathrm{Im}[fs_i(z_k)-fs_j(z_k)]|$

and the distance to all the other continuations are calculated according to:

$d_i = \sum_{j \neq i } \Delta_{i,j} $

With this distance measure, we introduce two criteria for selecting continuations.
For each continuation the two criteria are checked.
The mean distance $\tilde{d} = \frac{1}{\#d}\sum_i d_i$ is used for the first criterion.

1) The distance should be smaller than $0.95\tilde{d}$ 

2) The distance should be among the 51% lowest distances.

If both criteria are meet, the continuation is included. 
In the end, a simple average is done among the included continuations.
In case one wants to look at all the continuations and/or which of them are included in the averaging, it is also returned by the pade function.
     

In [10]:
import numpy as np

In [3]:
def acPade(z,fs,N,zout):
    '''
    # Input variables:
    # z    - complex. points in the complex plane.
    # fs   - complex. Corresponding (Green's) function in the complex plane. Columns represent different functions.
    # N    - int. Number of Padé coefficients to use
    # zout - complex. Points where continuation is evaluated
    # 
    # Assumes the asymptote of fs is 0+a/z+b/z^2. 
    '''
    if not N % 2 == 0 : 
        print "Error: The number of Padé coefficients in acPade function should be even."
    r = N/2
    M = len(z) # number of input points
    
    fout = []
    for f in fs.T: # column by column
        # Construct the matrix and rhs in linear system of equations. 
        y = f*z**r
        A = np.ones((M,N),dtype=np.complex)
        for i in range(M):
            A[i,:r] = z[i]**(np.arange(r)) 
            A[i,r:] = -f[i]*z[i]**(np.arange(r)) 
    
        # Calculated Padé coefficients
        sol = np.linalg.lstsq(A, y)
        x = sol[0] # Padé coefficents
        #print 'error_2= ',np.linalg.norm(np.dot(A,x)-y)
        #print 'residuals = ', sol[1]
        #print 'rank = ',sol[2]
        #print 'singular values / highest_singlular_value= ',sol[3]/sol[3][0]
    
        # Evaluate Padé approximant at points zout  
        numerator = np.zeros(len(zout),dtype=np.complex256)
        denomerator = np.zeros(len(zout),dtype=np.complex256)
        for i in range(r):
            numerator += x[i]*zout**i
            denomerator += x[r+i]*zout**i
        denomerator += zout**r
        fout.append(numerator/denomerator)
    fout = np.array(fout).T
    return fout

In [4]:
def pick_points(z,f,nmin,M,cols):
    '''
    z      - complex. Points in the complex plane.
    f      - complex. function values for different orbitals at the points 'z'.
    nmin   - integer. index of lowest points to pick.
    M      - integer. Number of points to pick.
    cols   - integer. Columns to pick.
    
    Pick out some of the input points from z and f.
    Pick out columns from f given by variable cols. 
    '''
    if nmin>=0 :
        zp = z[nmin:nmin+M]
        fp = f[nmin:nmin+M,cols]
    else:
        # use mirror symmetry of (Green's) function: f(z^*)_{i,i} = f(z)_{i,i}^*
        # Extension to multi-orbital case is: f(z^*)_{i,j} = f(z)_{j,i}^*   
        nadd = -nmin
        zp = np.hstack([np.conj(z[nadd-1::-1]),z[:M-nadd]])
        fp = np.vstack([np.conj(f[nadd-1::-1,cols]),f[:M-nadd,cols]])
    return zp,fp

## Pade function

In [5]:
def pade(zin,fin,zout):
    ''' 
    # Input variables:
    # zin  - complex. Points in complex plane.
    # fin  - complex. With several columns for different orbitals. Expects a 2d array
    # zout - complex. Points where continuation is evaluated.
    '''
    #---------------------------------------------------------------------------------------
    # Pade settings
    cols = [0] # range(17), [0,1] ,which columns in fin to continue 
    nmins = [-5]      # [-5,-1] , the minimum index of the input points to include. For value < 0 mirror values are added to input points before continuations. 
    Mmin = 40 # minimum number of input points to use
    Mmax = 80 # maximum number of input points to use
    Nmin = 10 # minimum number of Padé coefficients to use
    Nmax = 40 # maximum number of Padé coefficients to use
    Mstep = 4 # step size in M
    Nstep = 4 # step size in N
    diagonalPade = False # To perform continuations with only N==M or with N<=M  
    
    # Computational time:
    # O((Nmax-Nmin)*(Mmax-Mmin)*Nmax^3) for doing the continuations
    # O(((Nmax-Nmin)*(Mmax-Mmin))^2*Mmax) for calculating the devations between the continuations
    
    #---------------------------------------------------------------------------------------
    
    Ms = np.arange(Mmin,Mmax+1,Mstep)
    Ns = np.arange(Nmin,Nmax+1,Nstep)
    
    # 2Do: find constant asymptote term from input data to make the data to continue optimal for Pade 
    # It works quite well without this acually...
    
    # Loop over all the continuations to perform
    fs = [] 
    counter = 0
    for nmin in nmins:
        for M in Ms:
            #print 'M=',M
            if not diagonalPade:
                for N in Ns[Ns<=M]:
                    #print '  N=',N
                    counter += 1
                    zin_p,fin_p = pick_points(zin,fin,nmin,M,cols)
                    f = acPade(zin_p,fin_p,N,zout)
                    if np.all(np.imag(f) <= 0):
                        fs.append(f)
            else:
                counter += 1
                N = M
                zin_p,fin_p = pick_points(zin,fin,nmin,M,cols)
                f = acPade(zin_p,fin_p,N,zout)
                if np.all(np.imag(f) <= 0):
                    fs.append(f)
    print counter,' continuations performed per orbital.'
    fs = np.array(fs)
    masks = []
    fmeans = []
    for a in range(np.shape(fs)[2]): # Loop over the orbitals (they are independent from now on)
        f = fs[:,:,a]
        # loop over all physical continuations, calculating distance between imaginary part of the continuations
        delta = np.zeros((len(f),len(f))) # store distances between all the continuations
        for i,fi in enumerate(f):
            for j in range(i+1,len(f)):
                delta[i,j] = np.linalg.norm(fi.imag-f[j].imag,ord=1) 
        d = np.zeros(len(f)) # store accumulative distances to the other continuations
        for i,fi in enumerate(f):
            d[i] = np.sum(delta[:i,i]) + np.sum(delta[i,i+1:])
        # calculate if to include or reject continuations based on two criteria:
        c1 = d <= 0.95*np.mean(d)
        c2 = np.zeros(len(d),dtype=np.bool)
        ind = np.argsort(d)[:int(0.51*len(d))] # indices to 51% lowest distances 
        c2[ind] = True
        mask = c1 & c2 # continuations to include
        fmean = np.mean(f[mask],axis=0) # average selected continuations
        masks.append(mask)
        fmeans.append(fmean)
    
    masks = np.array(masks).T
    fmeans = np.array(fmeans).T
    
    return fmeans,fs,masks # the average continuations, all physical continuations and masks for averaging 

## Importing and preparing input data

In [36]:
# Non-interacting Hubbardmodel on the Bethe-lattice 
def G0bethe(z):
    W = 2 # bandwidth
    return 8*z/W**2*(1-np.sqrt(1-(2*z/W)**(-2)))
# Matsubara input data
beta = 100
zin = 1j*(2*np.arange(200)+1)*np.pi/beta
fin = np.atleast_2d(G0bethe(zin)).T
# Real axis 
w = np.linspace(-2,2,1000) # real axis energy
eim = 0.01 # distance above real axis
zout = w+eim*1j 

In [ ]:
# Sm7 data (these input files are not included in the project)
# Matsubara input data
#inputfolder = "/Users/johsc615/Dropbox/phd/projects/Pade/analytic_continuation_by_averaging_Pade_approximants/pics/Sm7-cluster/pek1/highPrecisionRaw/r1916_1shot_bis/type_01"
#import os
#os.chdir(inputfolder)
#fnameRe = 'real-sig-0103010100.dat'
#fnameIm = 'imag-sig-0103010100.dat'
#Gre = np.loadtxt(fnameRe)
#Gim = np.loadtxt(fnameIm)
#zin = Gre[:,0]*1j
#fin = Gre[:,1:]+Gim[:,1:]*1j
#print np.shape(fin)
# Real axis 
#w = np.linspace(-2,2,1000) # real axis energy
#eim = 0.01 # distance above real axis
#zout = w+eim*1j 

# RPA data (these input files are not included in the project)
# Matsubara input data
#inputfolder = "/Users/johsc615/Dropbox/phd/projects/bosonic_susceptibility/RPA-python"
#import os
#os.chdir(inputfolder)
#k = 40
#fin = np.loadtxt('output_mats_k'+str(k)+'.dat')
#zin = fin[:,0]*1j
#fin = fin[:,1]+fin[:,2]*1j
#fin = np.atleast_2d(fin).T
# Real axis 
#w = np.linspace(0.001,2,1000) # real axis energy
#eim = 0.05 # distance above real axis
#zout = w+eim*1j 

# Main function is called

In [41]:
fmeans,fs,masks = pade(zin,fin,zout)
print '--------------------  After pade routine  ------------------'
print  '(#physical continuation,#E,#orbitals) = ',np.shape(fs) 
print  '(#E,#orbitals) = ',np.shape(fmeans)
#print np.shape(fout)
#print fout.dtype 
#print 'mask=',mask

88  continuations performed per orbital.
--------------------  After pade routine  ------------------
(#physical continuation,#E,#orbitals) =  (75, 1000, 1)
(#E,#orbitals) =  (1000, 1)


### Plot spectrum

In [16]:
import matplotlib.pylab as plt

In [39]:
plt.figure(1)
plt.clf()
col = 0
plt.plot(w,-1/np.pi*np.imag(fs[:,:,col].T),'-g')  # all continuations 
plt.plot(w,-1/np.pi*np.imag(fs[masks[:,col],:,col].T),'-b')  # all picked continuations
plt.plot(w,-1/np.pi*np.imag(fmeans[:,col]),'-r',linewidth=3)  # the average result

plt.plot(w,-1/np.pi*np.imag(G0bethe(zout)),'-k',label='exact') # exact spectrum for bethe-lattice

plt.show()

### Save spectrum

In [40]:
tmp = np.vstack([w,(fmeans.real).T,(fmeans.imag).T]).T
np.savetxt('out.dat',tmp)

# Testing / debugging  of Padé routines
Space for exploring/improving the code